# 悪性URL検出

## urlからwhois情報を取得

python-whoisライブラリを使用
[PythonのWhoisライブラリの闇](https://qiita.com/hirachan/items/6787cb146838aade2d56)：他にもwhoisが取得できるライブラリはいくつかあるらしい
日本で使うにはpythonwhoisに変える必要があるかも

データセットとして以下のものを用いる

https://www.kaggle.com/datasets/sid321axn/malicious-urls-dataset?resource=download


- 良性URL: 428103個
- 改ざんURL:  96457個
- フィッシングURL: 94111個
- マルウェアURL: 32520個

の合計651,191個のデータセット

複数のデータセットを利用

- ISCX-URL-2016
- マルウェアドメインブラックリストのデータセット
- Phishtankデータセット
- PhishStormデータセット

In [14]:
# Googleドライブのマウント
from google.colab import drive
drive.mount('/content/drive')

import sys
sys.path.append('/content/drive/My Drive/Colab Notebooks')

%cd /content/drive/My Drive/Colab Notebooks

import torch
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

Mounted at /content/drive
/content/drive/My Drive/Colab Notebooks


# データセットの用意
- 上記のデータセットから、whoisライブラリを用いて40000個のurl分だけ取得する。

- whoisの取得に時間がかかりそうなので数を少なくしているが、出来れば全部使いたい

- JSONで与えられたwhoisの要素を、新しい特徴量として列に追加

毎回実行する必要はない

In [ ]:
!pip install python-whois pandas

In [ ]:
# whoisの取得
import pandas as pd
from concurrent.futures import ThreadPoolExecutor
import whois

data_path = "malicious_phish.csv"
df = pd.read_csv(data_path)
# 最初の40000個のURLを取得
urls = df['url'][:40000].tolist()
df

def fetch_whois(url):
    try:
        w = whois.whois(url)
        return str(w)
    except Exception as e:
        return str(e)

# 20スレッドを使用してWHOIS情報を並列に取得
with ThreadPoolExecutor(max_workers=20) as executor:
    whois_data = list(executor.map(fetch_whois, urls))

# 新しい列をデータフレームに追加
df.loc[:39999, 'whois_data'] = whois_data

# 新しいCSVファイルを保存
df.to_csv('add_whois40000.csv', index=False)
df
"""
# Whois情報の取得
whois_data = []
for url in urls:
    try:
        w = whois.whois(url)
        whois_data.append(w)
    except:
        whois_data.append(None)

# 新しい列の追加
df['whois_data'] = whois_data

# CSVファイルの保存
df.to_csv('add_whois.csv', index=False)
df
"""

In [ ]:
# csvの中身
import pandas as pd
data_path = "add_whois40000.csv"
df = pd.read_csv(data_path)
df

In [ ]:
# csvの要素数を減らす
import pandas as pd

# CSVファイルの読み込み
data_path = "add_whois40000.csv"
df = pd.read_csv(data_path, low_memory=False)

# 最初の40000件のデータを取り出す
df_subset = df.iloc[:40000]

# 新しいCSVファイルを保存
df_subset.to_csv('whois40000.csv', index=False)

In [ ]:
# JSONから特徴量を抽出して、新しい列として追加
import pandas as pd
import json

# CSVファイルを読み込む
df = pd.read_csv('whois40000.csv')

# 新しい列を作成する
for column in ["domain_name", "registrar", "whois_server", "referral_url", "updated_date", "creation_date", "expiration_date", "name_servers", "status", "emails", "dnssec", "name", "org", "address", "city", "state", "registrant_postal_code", "country"]:
    df[column] = None

# WHOIS列の各行から特徴量を抽出し、新しい列に値を設定する
for idx, row in df.iterrows():
    try:
        whois_data = json.loads(row['whois_data'])
        for key, value in whois_data.items():
            df.at[idx, key] = value
    except Exception as e:
      print(f"Error at index {idx}: {e}")


# 新しいCSVファイルとして保存する
df.to_csv('processed_whois40000.csv', index=False)



In [ ]:
# csvの中身 (列がなぜか大量に追加されていたので削除)
data_path = "processed_whois40000.csv"
df = pd.read_csv(data_path)
print(df.columns.tolist())
column_position = df.columns.get_loc('country')
print(f"The 'country' column is at position {column_position}")

df = df.iloc[:, :21]
print(df.iloc[:10, :20])